In [13]:
from qiskit.quantum_info import random_statevector
from scipy import sparse
import numpy
import re
from tqdm import tqdm
import scipy
import itertools
from qiskit.quantum_info import Pauli
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.operators.legacy import op_converter

The function below computes the inner product between two states $\langle\Psi|\Phi\rangle$.

In [14]:
 #This function computes the Inner Product
def InnerProd(A,B):
    IP=0.0
    for i in range(len(A)):
        for j in range(len(B)):
            if A['keys'][i]==B['keys'][j]:
                IP=IP+numpy.conjugate(A['vals'][i])*B['vals'][j]
#     hashedA = numpy.array([hash(s) for s in A['keys']])
#     hashedB = numpy.array([hash(s) for s in B['keys']])
#     AdjMat=numpy.equal.outer(hashedB, hashedA)
#     SparsedAdjMat=sparse.csr_matrix(AdjMat)
#     IP=numpy.sum(sparse.csr_matrix.dot(sparse.csr_matrix.dot(B['vals'],SparsedAdjMat),A['vals']))
    return IP

__Gram Schmidt Decomposition__<br>
If $|\phi_{0}\rangle,|\phi_{1}\rangle$ are non-orthogonal $\langle\phi_{0}|\phi_{1}\rangle\neq 0$<br>
then we can perform a Gram Schmidt decomposition as follows:<br>
$|\phi_{1,\perp}\rangle=\mathcal{N}^{-1}\left[|\phi_{1}\rangle-|\phi_{0}\rangle\langle\phi_{0}|\phi_{1}\rangle\right]$<br>
and we can check$:~$
$\langle\phi_{0}|\phi_{1,\perp}\rangle=0$

In [15]:
#This function carries out Gram Schmidt Orthonormalization
def GSOrth(PsiOld,PsiNonOrth):
    innerProd=InnerProd(PsiNonOrth,PsiOld)
    if(innerProd==0):
        return PsiNonOrth
    else:
        supCoeff=innerProd
        PsiUnionKeys=numpy.union1d(PsiOld['keys'],PsiNonOrth['keys'])
        PsiCompose=numpy.array([0.0j+numpy.sum(PsiNonOrth['vals'][PsiNonOrth['keys']==config])-supCoeff*numpy.sum(PsiOld['vals'][PsiOld['keys']==config]) for config in PsiUnionKeys],dtype='complex')
        PsiNew=numpy.rec.fromarrays([PsiUnionKeys,PsiCompose],names='keys,vals')
        return PsiNew
    

In [16]:
def per(n):
    List=[0]*int(2**n)
    for i in range(1<<n):
        s=bin(i)[2:]
        s='0'*(n-len(s))+s
        List[i]=s
    return List 

This function below allows us to decompose the statevector into two subspaces. One of them is composed of two qubits i,j in configurations $|00\rangle$, $|01\rangle$, $|10\rangle$, $|11\rangle$ and other qubits can be in some general configuration:<br>
$|\Psi\rangle=c_{0}|00\rangle|\phi_{0}\rangle+c_{1}|01\rangle|\phi_{1}\rangle+c_{2}|10\rangle|\phi_{2}\rangle+c_{3}|11\rangle|\phi_{3}\rangle$, note in general $\langle\phi_{i}|\phi_{j}\rangle\neq 0$.

In [17]:
def partitionStateVec(Psi,setPoints):
    #|\Psi\rangle=c_{0}|00>|\phi_{0}>+c_{1}|01>|\phi_{1}>+c_{2}|10>|\phi_{2}>+c_{3}|11>|\phi_{3}>
    manyBodyKet=numpy.array([numpy.array([conf.split(',')[0],eval(conf.split(',')[1])]) for conf in Psi],dtype='<U64')
    Ket=numpy.rec.fromarrays([manyBodyKet[:,0],manyBodyKet[:,1].astype(complex)],dtype=[('keys','<U64'),('vals','complex')])
    List=per(len(setPoints))
    BitConfigArr=['.'*setPoints[-1]+elem+'.'*(num_qubits-setPoints[-1]-len(setPoints)) for elem in List]
    r=[re.compile(BitConfig) for BitConfig in BitConfigArr]
    PsiSubSystemConfs=[[List[i],Ket[numpy.vectorize(lambda x:bool(r[i].match(x)))(Ket['keys'])]] for i in tqdm(range(len(List)))]
    PsiSubSystemConfs1=[]
    PsiUniqueConfs=[]
    for a in tqdm(range(len(List))):
        if((0 in PsiSubSystemConfs[a][1].shape)==False):
            tag=PsiSubSystemConfs[a][0]
            Psi=PsiSubSystemConfs[a][1]
            for conf in Psi['keys']:
                Psi['keys'][Psi['keys']==conf]=Psi['keys'][Psi['keys']==conf][0][0:setPoints[0]]+Psi['keys'][Psi['keys']==conf][0][setPoints[len(setPoints)-1]+1:num_qubits]
            sqrtIP=numpy.sqrt(InnerProd(Psi,Psi))
            Psi['vals']=(1/sqrtIP)*Psi['vals']
            ###modified
            IPs=numpy.array([abs(InnerProd(PsiSubSystemConfs1[j][1],Psi)+1) for j in range(len(PsiSubSystemConfs1))])
            sgn=1 if len(numpy.where(IPs<1e-8)[0])==0 else -1
            PsiSubSystemConfs1=PsiSubSystemConfs1+[[tag,Psi,sgn*sqrtIP]]
            cond=True
            for j in range(len(PsiUniqueConfs)):
                if(abs(InnerProd(PsiSubSystemConfs1[a][1],PsiUniqueConfs[j])-1)<1e-5):
                    cond=False
            if (cond is True):
                PsiUniqueConfs=PsiUniqueConfs+[PsiSubSystemConfs1[a][1]]
    uniqueTags=numpy.unique([conf[0] for conf in PsiSubSystemConfs1])
    return uniqueTags,PsiUniqueConfs,PsiSubSystemConfs1

In the state decomposed above the kets belonging to the subspaces need not be orthognal. This function below allows us to rewrite the statevector as a tensor decomposition between two subspaces A and B for a given partition, in which every ket in both the subspaces is orthogonal to every other ket.<br>
Mathematically we need a representation where,<br>
\begin{align}|\Psi\rangle=\sum_{i,j}D_{ij}|\psi_{A,i}\rangle|\psi_{B,j}\rangle, \langle\psi_{A,j}|\psi_{A,i}\rangle=\delta_{ij}$ and $\langle\psi_{B,j}|\psi_{B,i}\rangle=\delta_{ij}
\end{align}
This leads to a tensor network decomposition of the state.<br>
\begin{align}
|\Psi\rangle&=&\begin{pmatrix} |\psi_{A,1}\rangle & |\psi_{A,2}\rangle & |\psi_{A,3}\rangle & \ldots \end{pmatrix}\begin{pmatrix}D_{11} & D_{12} & D_{13} & \ldots\\
D_{21} & D_{22} & D_{23} &\ldots\\
D_{31} & D_{32} & D_{33} &\ldots\\
\vdots & \vdots& \vdots & \ddots
\end{pmatrix}\begin{pmatrix} |\psi_{A,1}\rangle \\|\psi_{A,2}\rangle \\ |\psi_{A,3}\rangle \\ \vdots
\end{pmatrix}
\end{align}

In [18]:
#Obtaining the D matrix for a given partition of two qubits from the rest
#D=  D_11  D_12  D_13 ...
#    D_21  D_22  D_23 ...
#    D_31  D_32  D_33 ...
#     .     .     .  .
#     .     .     .     .
#     .     .     .       .
def D(uniqueTags,PsiUniqueConfs,PsiSubSystemConfs):
    PsiOrtho=[PsiUniqueConfs[0]]
    for i in tqdm(range(1,len(PsiUniqueConfs))):
        Psi1=PsiUniqueConfs[i]
        innerProds=[InnerProd(PsiOrtho[j],Psi1) for j in range(len(PsiOrtho))]
        for j in range(len(PsiOrtho)):
            PsiUnionKeys=numpy.union1d(PsiOrtho[j]['keys'],Psi1['keys']) 
            PsiCompose=numpy.array([0.0j+numpy.sum(Psi1['vals'][Psi1['keys']==config])-innerProds[j]*numpy.sum(PsiOrtho[j]['vals'][PsiOrtho[j]['keys']==config]) for config in PsiUnionKeys],dtype='complex')
            Psi1=numpy.rec.fromarrays([PsiUnionKeys,PsiCompose],names='keys,vals')
        if len(numpy.where(numpy.array(Psi1['vals'])==0)[0])==len(Psi1['vals']):
            Psi1=[]
        else:    
            Psi1['vals']=Psi1['vals']/numpy.sqrt(InnerProd(Psi1,Psi1))
            Psi1=Psi1[numpy.abs(Psi1['vals'])>1e-5]
        A=numpy.array([numpy.abs(InnerProd(PsiOrtho[i],Psi1)) for i in range(len(PsiOrtho))]) 
        if len(numpy.where(numpy.abs(A)<1e-7)[0])==len(A) and len(Psi1)!=0:
            PsiOrtho=PsiOrtho+[Psi1]
        else:
            continue      
    boolCond=True
    for i in range(len(PsiOrtho)):
        for j in range(len(PsiOrtho)):
            if(i!=j):
                if(abs(InnerProd(PsiOrtho[i],PsiOrtho[j]))>1e-5):
                    boolCond=False
            else:
                if(abs(InnerProd(PsiOrtho[i],PsiOrtho[j])-1)>1e-5):
                    boolCond=False

    if(boolCond==True):
        print("Orthonormal basis formed")
    if(boolCond==False):
        A=[numpy.abs(numpy.abs(InnerProd(PsiOrtho[i],PsiOrtho[j]))-1) for i in range(len(PsiOrtho)) for j in range(len(PsiOrtho)) if i!=j]
        print(A)
        print("1-FAILED")
        return PsiOrtho 
    ###########################################
            #Computing the D matrix
    ############################################
    Dmat=numpy.zeros([len(uniqueTags),len(PsiOrtho)])*1j
    #extracting the tags associated with the two spin configurations 
    #within the members in Psi decomposed into subsystems->PsiSubSystemConfs
    tags=[psi[0] for psi in PsiSubSystemConfs]
    for i in range(len(uniqueTags)):
        for j in range(len(PsiOrtho)):
            ind=numpy.where(numpy.array(tags)==uniqueTags[i])[0]
            Sum=0
            for index in range(len(ind)):
                Sum=Sum+InnerProd(PsiSubSystemConfs[ind[index]][1],PsiOrtho[j])*PsiSubSystemConfs[ind[index]][2]
            Dmat[i][j]=Sum
    #Checking normalization from D matrix
    norm=numpy.dot(Dmat.flatten(),numpy.conjugate(Dmat.flatten()))
#     if(numpy.abs(norm-1)<1e-4):
#         print("D matrix constructed succesfully")
    if(numpy.abs(norm-1)>1e-4):
        print("D matrix construction failed",norm)
        return "2-FAILED",norm
#     else:
#         print("D matrix construction successful\n",D)
    return Dmat     

The function below carries out Schmidt decomposition of the entangled state $|\Psi\rangle$ for the subsystem partition two qubits and all rest.
Mathematically Schmidt decomposition is given by<br>
$|\Psi\rangle=\lambda_{1}|\psi_{1}\rangle|\chi_{1}\rangle+\lambda_{2}|\psi_{2}\rangle|\chi_{2}\rangle$, such that $S=[\lambda_{1},\lambda_{2}]$  are Schmidt coefficients and $\langle\psi_{2}|\psi_{1}\rangle=\langle\chi_{2}|\chi_{1}\rangle=0$<br>
We carry out below the Schmidt Decomposition via singular value decomposition of D matrix <br>
https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html<br>
$D=USV^{H}$, here $S$ is the Schmidt vector, U and V are 2d matrices

In [19]:
def SchmidtVector_partition(D):
    u,SchmidtVec,v=numpy.linalg.svd(D, full_matrices=True)
    return SchmidtVec

In [20]:
def Haar(num_qubits):
    #defining a random n qubit Haar random state
    state=random_statevector(2**num_qubits).data
    amplitudes=state
    bit_arr = [list(i) for i in itertools.product([0, 1], repeat=num_qubits)]
    bitStr_arr=[''.join([str(bit_arr[i][j]) for j in range(len(bit_arr[i]))]) for i in range(len(bit_arr))]
    #String rep. for the random state
    Psi=[bitStr_arr[i]+','+str(amplitudes[i]) for i in range(len(amplitudes))]
    return Psi,state

In [21]:
#note arange isolated qubits with indices in decreasing fashion
def SchmidtSpectrum(state,isolated_qbits):
    amplitudes=state
    bit_arr = [list(i) for i in itertools.product([0, 1], repeat=num_qubits)]
    bitStr_arr=[''.join([str(bit_arr[i][j]) for j in range(len(bit_arr[i]))]) for i in range(len(bit_arr))]
    #String rep. for the random state
    Psi=[bitStr_arr[i]+','+str(amplitudes[i]) for i in range(len(amplitudes))]
    uniqueTags,PsiUniqueConfs,PsiSubSystemConfs=partitionStateVec(Psi,isolated_qbits)
    Dmat=D(uniqueTags,PsiUniqueConfs,PsiSubSystemConfs)
    Schmidt_part=SchmidtVector_partition(Dmat)
    return Schmidt_part

In [10]:
# num_qubits=10
# Psi,state=Haar(10)

In [11]:
#setPoints=numpy.arange(int(num_qubits/2))[::-1]

In [12]:
# N=num_qubits
# uniqueTags,PsiUniqueConfs,PsiSubSystemConfs=partitionStateVec(Psi,setPoints)
# # # #constructing D matrix
# Dmat=D(uniqueTags,PsiUniqueConfs,PsiSubSystemConfs)
# norm=numpy.dot(Dmat.flatten(),numpy.conjugate(Dmat.flatten()))
# # # #Schmidt vector for the partition qubit pair (i,j) from rest
# Schmidt_part=SchmidtVector_partition(Dmat)